# Analyst Investigation View Builder

## Phase
Phase 2 — SOC Operational Workflow

## Objective
Generate investigation-ready case summaries from modeled SOC incidents.


In [1]:
import pandas as pd
from pathlib import Path


In [2]:
PROJECT_ROOT = Path(r"D:\soc-dashboard-suite-main\soc-dashboard-suite-main")

INCIDENTS_PATH = PROJECT_ROOT / "data" / "enriched" / "soc_incidents.csv"
ALL_ALERTS_PATH = PROJECT_ROOT / "data" / "enriched" / "alerts_scored_prioritized.csv"

incident_df = pd.read_csv(INCIDENTS_PATH, parse_dates=["start_time", "end_time"])
email_alerts = pd.read_csv(ALL_ALERTS_PATH, parse_dates=["event_time"])

incident_df.head()


,incident_id,user_id,start_time,end_time,alert_count,sources_involved
0,INC0001,user1,2025-01-17 00:42:00,2025-01-18 17:13:00,12,authentication
1,INC0002,user1,2025-01-19 08:26:00,2025-01-20 17:14:00,6,authentication
2,INC0003,user1,2025-01-21 09:44:00,2025-01-23 08:21:00,5,authentication
3,INC0004,user1,2025-01-23 09:52:00,2025-01-24 16:07:00,3,authentication
4,INC0005,user1,2025-01-25 10:53:00,2025-01-27 09:27:00,4,authentication


In [3]:
def classify_incident(row):
    if "correlated" in row["sources_involved"]:
        return "critical"
    elif row["alert_count"] >= 5:
        return "high"
    elif row["alert_count"] >= 2:
        return "medium"
    else:
        return "low"

incident_df["incident_severity"] = incident_df.apply(classify_incident, axis=1)


In [4]:
incident_df["investigation_summary"] = (
    "User " + incident_df["user_id"].astype(str) +
    " triggered " + incident_df["alert_count"].astype(str) +
    " alerts involving: " + incident_df["sources_involved"] +
    ". Severity classified as " + incident_df["incident_severity"] + "."
)


In [6]:
def recommend_action(severity):
    if severity == "critical":
        return "Immediate account lock + credential reset + forensic review"
    elif severity == "high":
        return "Force password reset and review login history"
    elif severity == "medium":
        return "Monitor account and validate user activity"
    else:
        return "No immediate action required"

incident_df["recommended_action"] = incident_df["incident_severity"].apply(recommend_action)


In [9]:
investigation_view = incident_df[[
    "incident_id",
    "user_id",
    "start_time",
    "end_time",
    "alert_count",
    "incident_severity",
    "investigation_summary",
    "recommended_action"
]]

investigation_view


,incident_id,user_id,start_time,end_time,alert_count,incident_severity,investigation_summary,recommended_action
0,INC0001,user1,2025-01-17 00:42:00,2025-01-18 17:13:00,12,high,User user1 triggered 12 alerts involving: auth...,Force password reset and review login history
1,INC0002,user1,2025-01-19 08:26:00,2025-01-20 17:14:00,6,high,User user1 triggered 6 alerts involving: authe...,Force password reset and review login history
2,INC0003,user1,2025-01-21 09:44:00,2025-01-23 08:21:00,5,high,User user1 triggered 5 alerts involving: authe...,Force password reset and review login history
3,INC0004,user1,2025-01-23 09:52:00,2025-01-24 16:07:00,3,medium,User user1 triggered 3 alerts involving: authe...,Monitor account and validate user activity
4,INC0005,user1,2025-01-25 10:53:00,2025-01-27 09:27:00,4,medium,User user1 triggered 4 alerts involving: authe...,Monitor account and validate user activity
...,...,...,...,...,...,...,...,...
298,INC0299,user9,2025-01-19 16:59:00,2025-01-21 01:44:00,6,high,User user9 triggered 6 alerts involving: authe...,Force password reset and review login history
299,INC0300,user9,2025-01-22 04:14:00,2025-01-24 00:55:00,9,high,User user9 triggered 9 alerts involving: authe...,Force password reset and review login history
300,INC0301,user9,2025-01-24 05:41:00,2025-01-25 18:50:00,7,high,User user9 triggered 7 alerts involving: authe...,Force password reset and review login history
301,INC0302,user9,2025-01-26 14:27:00,2025-01-28 13:07:00,8,high,User user9 triggered 8 alerts involving: authe...,Force password reset and review login history


In [8]:
OUTPUT_PATH = PROJECT_ROOT / "data" / "enriched" / "soc_investigation_view.csv"
investigation_view.to_csv(OUTPUT_PATH, index=False)

print("Saved investigation view to:", OUTPUT_PATH)


Saved investigation view to: D:\soc-dashboard-suite-main\soc-dashboard-suite-main\data\enriched\soc_investigation_view.csv
